In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [2]:
import pandas as pd

# Importing data

In [3]:
raw_data = pd.read_csv("../data/train.csv")

In [4]:
clean_data = raw_data.drop(["Cabin", "Name", "PassengerId", "Ticket"], axis=1)

In [5]:
clean_data = pd.get_dummies(clean_data).fillna(-1)

In [6]:
train, test = train_test_split(clean_data)

In [7]:
def split_x_y(dataframe, target):
    return dataframe.drop(target, axis=1), dataframe[target]

In [8]:
X_train, y_train = split_x_y(train, "Survived")
X_test, y_test = split_x_y(test, "Survived")

# Baseline model

In [9]:
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=-1, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [10]:
y_pred = rf.predict_proba(X_test)

In [11]:
auc = roc_auc_score(y_true=y_test, y_score=y_pred[:, 1])

In [12]:
auc

0.81840034965034958

# Deep Forest

## By Hand

In [13]:
from sklearn.model_selection import StratifiedKFold

In [14]:
rf1 = RandomForestClassifier(n_estimators=100, n_jobs=-1, max_depth=4)
rf2 = RandomForestClassifier(n_estimators=100, n_jobs=-1, max_depth=10)

In [15]:
rf1.fit(X_train, y_train)
y_pred_train_1 = rf1.predict_proba(X_train)
y_pred_test_1 = rf1.predict_proba(X_test)

y_pred_train_1 = pd.DataFrame(y_pred_train_1, columns=["rf1_0", "rf1_1"], index=X_train.index)
y_pred_test_1 = pd.DataFrame(y_pred_test_1, columns=["rf1_0", "rf1_1"], index=X_test.index)

In [16]:
rf2.fit(X_train, y_train)
y_pred_train_2 = rf2.predict_proba(X_train)
y_pred_test_2 = rf2.predict_proba(X_test)

y_pred_train_2 = pd.DataFrame(y_pred_train_2, columns=["rf2_0", "rf2_1"], index=X_train.index)
y_pred_test_2 = pd.DataFrame(y_pred_test_2, columns=["rf2_0", "rf2_1"], index=X_test.index)

In [17]:
hidden_train_1 = pd.concat([X_train, y_pred_train_1, y_pred_train_2], axis=1)
hidden_test_1 = pd.concat([X_test, y_pred_test_1, y_pred_test_2], axis=1)

In [18]:
hidden_train_1.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,rf1_0,rf1_1,rf2_0,rf2_1
542,3,11.0,4,2,31.2750,1,0,0,0,1,0.688593,0.311407,0.928719,0.071281
853,1,16.0,0,1,39.4000,1,0,0,0,1,0.106854,0.893146,0.000000,1.000000
496,1,54.0,1,0,78.2667,1,0,1,0,0,0.057463,0.942537,0.000000,1.000000
506,2,33.0,0,2,26.0000,1,0,0,0,1,0.128737,0.871263,0.001538,0.998462
557,1,-1.0,0,0,227.5250,0,1,1,0,0,0.619901,0.380099,0.966940,0.033060


In [19]:
rf3 = RandomForestClassifier(n_estimators=300, n_jobs=-1)
rf3.fit(hidden_train_1, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=300, n_jobs=-1, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [20]:
y_pred3 = rf3.predict_proba(hidden_test_1)

In [21]:
roc_auc_score(y_test, y_pred3[:, 1])

0.81215034965034971

## With API

In [22]:
from deepforest.layer import Layer, InputLayer

In [23]:
input_layer = InputLayer(RandomForestClassifier(n_estimators=100, n_jobs=-1, max_depth=4),
                         RandomForestClassifier(n_estimators=100, n_jobs=-1, max_depth=10))

In [24]:
hidden_layer = Layer(input_layer,
                     RandomForestClassifier(n_estimators=50, n_jobs=-1, max_depth=4),
                     RandomForestClassifier(n_estimators=50, n_jobs=-1, max_depth=10))

In [26]:
hidden_layer.fit(X_train, y_train)

In [27]:
hidden_layer.predict(X_test)

,model0_0,model0_1,model1_0,model1_1
602,0.691874,0.308126,0.722429,0.277571
878,0.982569,0.017431,1.000000,0.000000
821,0.970659,0.029341,0.979765,0.020235
115,0.972282,0.027718,0.999765,0.000235
489,0.090206,0.909794,0.100000,0.900000
588,0.972053,0.027947,0.999765,0.000235
879,0.000896,0.999104,0.000000,1.000000
203,0.959542,0.040458,0.979765,0.020235
210,0.837261,0.162739,0.599286,0.400714
309,0.001454,0.998546,0.000000,1.000000
